In [ ]:
def loading():

    # Classification Model loading
    import tensorflow as tf
    from tensorflow.keras.models import load_model
    CLASSIFICATION_MODEL_PATH = "/content/drive/MyDrive/final_model_all_classes.keras"

    # Load the saved model with custom layer
    @tf.keras.utils.register_keras_serializable(package="custom_layers")
    class HistogramLayer(tf.keras.layers.Layer):
        def __init__(self, nbins=256, value_range=(0.0, 255.0), **kwargs):
            super().__init__(**kwargs)
            self.nbins = int(nbins)
            self.value_range = (float(value_range[0]), float(value_range[1]))

        def call(self, inputs):
            x = tf.cast(inputs, tf.float32)
            gray = tf.image.rgb_to_grayscale(x)

            def per_image(img):
                img = tf.reshape(img, [-1])
                hist = tf.histogram_fixed_width(img, self.value_range, nbins=self.nbins)
                hist = tf.cast(hist, tf.float32)
                s = tf.reduce_sum(hist)
                return tf.cond(s > 0, lambda: hist / s, lambda: hist)

            return tf.map_fn(per_image, gray, fn_output_signature=tf.float32)

        def get_config(self):
            cfg = super().get_config()
            cfg.update({"nbins": self.nbins, "value_range": self.value_range})
            return cfg

    image_classifier = load_model(CLASSIFICATION_MODEL_PATH, compile=False, custom_objects={'HistogramLayer': HistogramLayer})
    return image_classifier


    #architecture of fcc convertor style_transformer

    class PatchEmbed(nn.Module):
        def __init__(self, in_ch, emb, patch):
            super().__init__()
            self.proj = nn.Conv2d(in_ch, emb, kernel_size=patch, stride=patch)
        def forward(self, x):
            x = self.proj(x)
            B, E, H, W = x.shape
            tokens = x.flatten(2).transpose(1,2)
            return tokens, (H, W)

    class PatchUnembed(nn.Module):
            def __init__(self, emb, out_ch, patch):
                super().__init__()
                self.deproj = nn.ConvTranspose2d(emb, out_ch, kernel_size=patch, stride=patch)
            def forward(self, tokens, hw):
                B, N, E = tokens.shape
                H, W = hw
                x = tokens.transpose(1,2).reshape(B, E, H, W)
                return self.deproj(x)

    class StyleTransformer(nn.Module):
            def __init__(self, in_ch=3, out_ch=3, emb=96, layers=3, heads=6, ff=256, patch=16, image_size=256):
                super().__init__()
                self.image_size = image_size
                self.patch = patch
                self.h_p = image_size // patch
                self.w_p = self.h_p

                self.encoder = nn.Sequential(
                    nn.Conv2d(in_ch, 64, kernel_size=3, padding=1),
                    nn.ReLU(inplace=True),
                    nn.Conv2d(64, emb, kernel_size=3, padding=1),
                    nn.ReLU(inplace=True),
                )
                self.patch_embed = PatchEmbed(emb, emb, patch)
                encoder_layer = nn.TransformerEncoderLayer(d_model=emb, nhead=heads,
                                                        dim_feedforward=ff, dropout=0.1,
                                                        activation='gelu', batch_first=True)
                self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=layers)
                self.patch_unembed = PatchUnembed(emb, emb, patch)
                self.decoder = nn.Sequential(
                    nn.Conv2d(emb, 64, kernel_size=3, padding=1),
                    nn.ReLU(inplace=True),
                    nn.Conv2d(64, out_ch, kernel_size=3, padding=1),
                    nn.Sigmoid()
                )
                N = (self.h_p * self.w_p)
                self.pos_embed = nn.Parameter(torch.randn(1, N, emb) * 0.02)

            def forward(self, x):
                B, C, H, W = x.shape
                assert H == self.image_size and W == self.image_size, f"Expected {self.image_size}x{self.image_size}, got {H}x{W}"
                z = self.encoder(x)
                tokens, hw = self.patch_embed(z)
                tokens = tokens + self.pos_embed.to(tokens.device)
                out_tokens = self.transformer(tokens)
                z_hat = self.patch_unembed(out_tokens, hw)
                z_hat = F.interpolate(z_hat, size=(H, W), mode='bilinear', align_corners=False)
                out = self.decoder(z_hat)
                return out

    def load_fcc_model(weights_path="fcc_best_model.pt", device="cuda"):
        model = StyleTransformer(image_size=256)
        state_dict = torch.load(weights_path, map_location=device)
        
        if isinstance(state_dict, dict) and "model_state" in state_dict:
            state_dict = state_dict["model_state"]
        
        model.load_state_dict(state_dict)
        model.to(device).eval()
        return model

    fcc_model = load_fcc_model()


    from unsloth import FastVisionModel
    import torch
    from transformers import AutoModelForCausalLM, AutoModel, AutoTokenizer
    from peft import PeftModel

    QWEN_model, QWEN_tokenizer = FastVisionModel.from_pretrained(
        "unsloth/Qwen3-VL-32B-Instruct-unsloth-bnb-4bit",
        load_in_4bit=True,                   # Use 4bit quantization
        use_gradient_checkpointing="unsloth" # Enables long context with low VRAM
    )

    return QWEN_model
    return QWEN_tokenizer


    # 3 lora adapters, put paths in empty slots

    import torch

    #SAR LORA
    sar_adapter_path = ""
    lora_adapter_sar = torch.load(sar_adapter_path)

    #OPTICAL LORA
    optical_adapter_path = ""
    lora_adapter_optical = torch.load(optical_adapter_path)

    #YOLO LORA
    yolo_adapter_path = ""
    lora_adapter_yolo = torch.load(yolo_adapter_path)
    
    return lora_adapter_sar, lora_adapter_optical, lora_adapter_yolo


    #GEOGROUND

    from huggingface_hub import snapshot_download
    import os

    model_dir = "geoground_model"

    # Remove old incomplete downloads
    if os.path.exists(model_dir):
        import shutil
        shutil.rmtree(model_dir)


    os.makedirs(model_dir, exist_ok=True)

    # Download with better error handling
    snapshot_download(
        repo_id="erenzhou/GeoGround",
        local_dir=model_dir,
        local_dir_use_symlinks=False,
        resume_download=True,
        max_workers=4
    )

    actual_model_dir = os.path.join(model_dir, "llava-v1.5-7b-task-lora-geoground")

    # ---------------------- LLaVA SETUP (SCRIPT SAFE) ----------------------
    import os
    import sys
    import subprocess

    def run_cmd(cmd):
        """Run a shell command cleanly."""
        print(f"$ {cmd}")
        subprocess.run(cmd, shell=True, check=True)

    # 1. Clone repository if not already present
    if not os.path.exists("LLaVA"):
        run_cmd("git clone https://github.com/haotian-liu/LLaVA.git")

    # 2. Add LLaVA folder to python path
    sys.path.insert(0, os.path.abspath("LLaVA"))

    # 3. Install LLaVA in editable mode WITHOUT dependencies
    old_path = os.getcwd()
    os.chdir("LLaVA")

    run_cmd("pip install -e . --no-deps")

    os.chdir(old_path)

    # ------------------------------------------------------------------------